# Parser

In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import os

In [ ]:
### MY IDEA --> read the txt file with the url
#with open('master_urls.txt', 'r') as file:
    # Read the content of the file
#    content = file.read()

# Split the content into a list of URLs
#url_list = content.split('\n')

# Print the list of URLs
#for url in url_list:
#    print(url)

In [19]:
#msc_page_url = "https://www.findamasters.com/masters-degrees/course/addictions-msc/?i132d4318c27100"

#msc_page_url = ["https://www.findamasters.com" + element if not element.startswith("http") else element for element in url_list]


#DEFINE THE FUNCTION TO EXTRACT INFORMATION

def extract_msc_page(msc_page_url):
  
  contents = {'url': "https://www.findamasters.com" + msc_page_url}

  page_url = requests.get(msc_page_url)
  page_soup = BeautifulSoup(page_url.text)


  #course name
  page_links = page_soup.find('h1', {'class': 'text-white course-header__course-title'})
  name = page_links.get_text() 
  contents["courseName"] = name

  #university name
  page_links = page_soup.find_all('a', {'class': 'course-header__institution'})
  contents['universityName'] = page_links[0].contents[0] 

  #faculty name
  page_links = page_soup.find_all('a', {'class': 'course-header__department'})
  contents['facultyName'] = page_links[0].contents[0] 

  #full time
  try:
    page_links= page_soup.find('a', {'class':'inheritFont concealLink text-decoration-none text-gray-600'})
    time = page_links.get_text()
    contents['isItFullTime'] = time
  except AttributeError:
    contents['isItFullTime'] = ""

  #description
  page_links = page_soup.find('div', {'id': 'Snippet'})
  description = page_links.get_text() 
  contents["description"] = description

  #starting date
  page_links = page_soup.find('span', {'class': 'key-info__content key-info__start-date py-2 pr-md-3 text-nowrap d-block d-md-inline-block'})
  starting = page_links.get_text() 
  contents["startDate"] = starting

  #fees
  page_links = page_soup.find('div', {'class': 'course-sections course-sections__fees tight col-xs-24'})
  fees = page_links.get_text() 
  contents["fees"] = fees

  #modality
  try:
    page_links = page_soup.find('a', {'title': 'View all MSc courses'})
    modality = page_links.get_text() 
    contents["modality"] = modality
  except AttributeError:
    contents['modality'] = ""

  #Duration
  page_links = page_soup.find('span', {'class': 'key-info__content key-info__duration py-2 pr-md-3 d-block d-md-inline-block'})
  duration = page_links.get_text() 
  contents["duration"] = duration

  #city
  try:
    page_links = page_soup.find('a', {'class': 'card-badge text-wrap text-left badge badge-gray-200 p-2 m-1 font-weight-light course-data course-data__city'})
    city = page_links.get_text() 
    contents["city"] = city
  except AttributeError:
    contents['modality'] = ""

  #country
  try:
    page_links = page_soup.find('a', {'class': 'card-badge text-wrap text-left badge badge-gray-200 p-2 m-1 font-weight-light course-data course-data__country'})
    country = page_links.get_text() 
    contents["country"] = country
  except AttributeError:
    contents['modality'] = ""

  #administration
  try:
    page_links = page_soup.find('a', {'class': 'card-badge text-wrap text-left badge badge-gray-200 p-2 m-1 font-weight-light course-data course-data__on-campus'})
    administration = page_links.get_text() 
    contents["administration"] = administration
  except AttributeError:
        contents["administration"] = ""
    
  return contents


In [8]:

def extract_msc_page_from_html(html_content):
    contents = {}

    # Parse HTML content
    page_soup = BeautifulSoup(html_content, 'html.parser')

    # URL
    contents['url'] = "https://www.findamasters.com"

    # Course name
    try:
        page_links = page_soup.find('h1', {'class': 'text-white course-header__course-title'})
        name = page_links.get_text()
        contents["courseName"] = name.strip()
    except AttributeError:
        contents['courseName'] = ""

    # University name
    try:
        page_links = page_soup.find_all('a', {'class': 'course-header__institution'})
        contents['universityName'] = page_links[0].contents[0].strip()
    except (AttributeError, IndexError):
        contents['universityName'] = ""

    # Faculty name
    try:
        page_links = page_soup.find_all('a', {'class': 'course-header__department'})
        contents['facultyName'] = page_links[0].contents[0].strip()
    except (AttributeError, IndexError):
        contents['facultyName'] = ""

    # Full time
    try:
        page_links = page_soup.find('a', {'class': 'inheritFont concealLink text-decoration-none text-gray-600'})
        time = page_links.get_text().strip()
        contents['isItFullTime'] = time
    except AttributeError:
        contents['isItFullTime'] = ""

    # Description
    try:
        page_links = page_soup.find('div', {'id': 'Snippet'})
        description = page_links.get_text().strip()
        contents["description"] = description
    except AttributeError:
        contents['description'] = ""

    # Starting date
    try:
        page_links = page_soup.find('span', {'class': 'key-info__content key-info__start-date py-2 pr-md-3 text-nowrap d-block d-md-inline-block'})
        starting = page_links.get_text().strip()
        contents["startDate"] = starting
    except AttributeError:
        contents['startDate'] = ""

    # Fees
    try:
        page_links = page_soup.find('div', {'class': 'course-sections course-sections__fees tight col-xs-24'})
        fees = page_links.get_text().strip()
        contents["fees"] = fees
    except AttributeError:
        contents['fees'] = ""

    # Modality
    try:
        page_links = page_soup.find('a', {'title': 'View all MSc courses'})
        modality = page_links.get_text().strip()
        contents["modality"] = modality
    except AttributeError:
        contents['modality'] = ""

    # Duration
    try:
        page_links = page_soup.find('span', {'class': 'key-info__content key-info__duration py-2 pr-md-3 d-block d-md-inline-block'})
        duration = page_links.get_text().strip()
        contents["duration"] = duration
    except AttributeError:
        contents['duration'] = ""

    # City
    try:
        page_links = page_soup.find('a', {'class': 'card-badge text-wrap text-left badge badge-gray-200 p-2 m-1 font-weight-light course-data course-data__city'})
        city = page_links.get_text().strip()
        contents["city"] = city
    except AttributeError:
        contents['city'] = ""

    # Country
    try:
        page_links = page_soup.find('a', {'class': 'card-badge text-wrap text-left badge badge-gray-200 p-2 m-1 font-weight-light course-data course-data__country'})
        country = page_links.get_text().strip()
        contents["country"] = country
    except AttributeError:
        contents['country'] = ""

    # Administration
    try:
        page_links = page_soup.find('a', {'class': 'card-badge text-wrap text-left badge badge-gray-200 p-2 m-1 font-weight-light course-data course-data__on-campus'})
        administration = page_links.get_text().strip()
        contents["administration"] = administration
    except AttributeError:
        contents["administration"] = ""

    return contents


#with open("html_pages/page_1/3d-design-for-virtual-environments-msc.html", "r", encoding="utf-8") as file:
#    html_content = file.read()

#result = extract_msc_page_from_html(html_content)
#print(result)




In [ ]:
folder_path = 'html_pages/'
for n in range(1, 401):
    file_name = f'page_{n}'
    path = os.path.join(folder_path, file_name)
    
    # Check if the file exists before attempting to open it
    if os.path.exists(path):
        print(path)
        with open(path, 'r', encoding='utf-8') as file:
            html_content = file.read()
            extract_msc_page_from_html(html_content)
    else:
        print(f"File not found: {path}")

In [ ]:
folder_path = 'html_pages/'

result_df = pd.DataFrame()
# Walk through the directory and its subdirectories
for foldername, subfolders, filenames in os.walk(folder_path):
    # Iterate through all the files in the current subdirectory
    for filename in filenames:
        # Construct the file path
        path = os.path.join(foldername, filename)
        
        # Check if the file exists before attempting to open it
        if os.path.exists(path):
            # Print the file path
            print(path)
            
            # Open the file in read mode with UTF-8 encoding
            with open(path, 'r', encoding='utf-8') as file:
                # Read the HTML content of the file
                html_content = file.read()
                
                # Call the function to extract information from the HTML content
                result_dict = extract_msc_page_from_html(html_content)
                result_df = result_df.append(result_dict, ignore_index=True)


        else:
            # Print a message if the file is not found
            print(f"File not found: {path}")

result_df.to_json('html_pages.json', orient='records', lines=True)

In [38]:
result_df['fees'] = result_df['fees'].str.replace('Fees', '')

In [40]:
output_directory = "tsv_files/"

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Iterate through each row and create a TSV file
for index, row in result_df.iterrows():
    file_name = f"course_{index}.tsv"
    file_path = os.path.join(output_directory, file_name)

    # Extract relevant columns and write to the TSV file
    selected_columns = ['courseName', 'universityName', 'facultyName', 'isItFullTime', 'description', 'startDate',
                        'fees','modality','duration','city','country','administration', 'url']
    selected_data = row[selected_columns]
    selected_data.to_csv(file_path, sep='\t', index=False, header=False)

print("TSV files created successfully.")

TSV files created successfully.
